In [1]:
# Load Libraries

import os
import joblib
from datetime import datetime
import glob
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error, mean_absolute_error
from pmdarima import auto_arima


In [2]:
# Load combined dataset

# file directory
file= "../data/Air_Quality_Lagos_Combined.csv"

# load into DataFrame
df = pd.read_csv(file, index_col="date", parse_dates=True)

# resampling the data to 1 hour interval
y = df["PM2.5"].resample('6h').mean().interpolate(method = "time")

y.head()

date
2023-11-01 12:00:00+01:00    19.000000
2023-11-01 18:00:00+01:00    15.610000
2023-11-02 00:00:00+01:00    19.867137
2023-11-02 06:00:00+01:00    24.124275
2023-11-02 12:00:00+01:00    28.381412
Freq: 6h, Name: PM2.5, dtype: float64

In [ ]:
# Size of the Dataset
shape = y.shape

print(f"The Shape of the Dataset is: {shape}")
print(f"The Dataset has: {shape[0]} historical data")
print(f"The number of missing data is: {y.isna().sum()}")
# print(f"The dataset have {shape[1]} column. For PM2.5 Readings")

In [ ]:
# Data Split 
cutt_off = int(len(y) * 0.8)

train = y.iloc[:cutt_off]
test = y.iloc[cutt_off:]


print(f"Train split: {train.shape}")
print(f"Test split: {test.shape}")

# Correct HyperParameter

In [ ]:
model = auto_arima(
    train,
    seasonal=True,
    m=28,  # season length (e.g., 4 for 6-hourly data = 1 day)
    trace=True,
    error_action='ignore',
    suppress_warnings=True
)

# AR (AutoRegressive) Model

In [ ]:
# Fit the AR model
ar_model = AutoReg(train, lags=29)  # for 24hours or 1 day
ar_result = ar_model.fit()

In [ ]:
# Model summary
ar_result.summary()

In [ ]:
# Forecast
y_ar_pred = ar_result.predict(test.index.min(), test.index.max())
y_ar_pred.head()


In [ ]:
# Baseline Model Metrics
mean = train.mean()
y_mean_pred = len(train) * [mean]

# MSE and MAE of the Baseline model
mse_baseline = mean_squared_error(train, y_mean_pred)
mae_baseline = mean_absolute_error(train, y_mean_pred)


print(f"MSE of the baseline model: {round(mse_baseline, 2)}")
print(f"MAE of the baseline model: {round(mae_baseline, 2)}")

In [ ]:
# Evaluation
mse = mean_squared_error(test, y_ar_pred)
mae = mean_absolute_error(test, y_ar_pred)

print(f"AR Model - MSE: {mse:.2f}")
print(f"AR Model - MAE: {mae:.2f}")

In [ ]:
# Residual graph
fig, ax = plt.subplots(figsize = (15, 5))
ar_result.resid.plot(ax = ax)
plt.title("Residual Plot")
plt.xlabel("Date")
plt.show()

In [ ]:
# Residual Histogram
ar_result.resid.plot(
    kind = "hist", 
    xlabel = "Residual",
    ylabel = "frequency",
    title = "Residual Distribution"
)

In [ ]:
# DataFrame for Actual Test data and Predictions

df_pred_test = pd.DataFrame({
            'y_test':test,
            'y_pred':y_ar_pred
}, index = test.index)
df_pred_test.tail()

In [ ]:
# Plot results
fig = px.line(
    data_frame= df_pred_test,
    title = "Prediction vs Actual Test data ",
    labels= {"value" : "PM2.5"}
)
fig.show()

In [ ]:
# Walk Forward Validation

y_pred_wfv = pd.Series()
history = train.copy()
for i in range(len(test)):
    model = AutoReg(history, lags=29).fit()
    next_pred = model.forecast()
    y_pred_wfv = pd.concat([y_pred_wfv, next_pred])
    history = pd.concat([history, test[next_pred.index]]) 


In [ ]:
df_pred_wfv = pd.DataFrame({
            'y_test':test,
            'y_pred':y_pred_wfv
}, index = test.index)
df_pred_wfv.tail()

In [ ]:
fig = px.line(
    data_frame= df_pred_wfv,
    title = "Walk forward Validation Prediction and Actual Test data",
    labels= {"value" : "PM2.5"}
)

fig.show()

In [ ]:
# Evaluation of the Walk forward Validation Prediction

mse = mean_squared_error(test, y_pred_wfv)
mae = mean_absolute_error(test, y_pred_wfv)

print(f"AR Model - MSE: {mse:.2f}")
print(f"AR Model - MAE: {mae:.2f}")

## Summary

The AR model Underperformed. why:
- The forecast (red line) is almost flat, failing to capture the volatility, spikes, and seasonality in the test set.
- AR models rely only on past values of the series (lags) and assume a linear relationship, which often fails when: There’s strong seasonality or nonlinearity.
- Residuals are not white noise, as we saw earlier from decomposition and ACF plots.



# ARMA (AutoRegressive Moving Average) Model

In [ ]:
# Fit ARMA(p=2, q=2) on differenced data (you can tune p and q later)
arma_model = ARIMA(train, order=(2, 0, 0))  # d=0 because we assume stationary (you can test d=1 if needed)
arma_result = arma_model.fit()

In [ ]:
arma_result.summary()

In [ ]:
# Forecast
y_arma_pred = arma_result.predict(test.index.min(), test.index.max())
y_arma_pred.head()

In [ ]:
# Baseline Model Metrics
mean = train.mean()
y_mean_pred = len(train) * [mean]

# MSE and MAE of the Baseline model
mse_baseline = mean_squared_error(train, y_mean_pred)
mae_baseline = mean_absolute_error(train, y_mean_pred)


print(f"MSE of the baseline model: {round(mse_baseline, 2)}")
print(f"MAE of the baseline model: {round(mae_baseline, 2)}")

In [ ]:
# Evaluation
mse = mean_squared_error(test, y_arma_pred)
mae = mean_absolute_error(test, y_arma_pred)

print(f"ARMA Model - MSE: {mse:.2f}")
print(f"ARMA Model - MAE: {mae:.2f}")

In [ ]:
# Residual graph
fig, ax = plt.subplots(figsize = (15, 5))
arma_result.resid.plot(ax = ax)
plt.title("Residual Plot")
plt.xlabel("Date")
plt.show()

In [ ]:
# Residual Histogram
arma_result.resid.plot(
    kind = "hist", 
    xlabel = "Residual",
    ylabel = "frequency",
    title = "Residual Distribution"
)

In [ ]:
# DataFrame for Actual Test data and Predictions

df_pred_test = pd.DataFrame({
            'y_test':test,
            'y_pred':y_arma_pred
}, index = test.index)
df_pred_test.tail()

In [ ]:
# Plot results
fig = px.line(
    data_frame= df_pred_test,
    title = "Prediction vs Actual Test data ",
    labels= {"value" : "PM2.5"}
)
fig.show()

In [ ]:
# Walk Forward Validation

y_pred_wfv = pd.Series()
history = train.copy()
for i in range(len(test)):
     model = ARIMA(history, order = (3, 1, 1)).fit()
     next_pred = model.forecast()
     y_pred_wfv = pd.concat([y_pred_wfv, next_pred])
     history = pd.concat([history, test[next_pred.index]])


In [ ]:
df_pred_wfv = pd.DataFrame({
            'y_test':test,
            'y_pred':y_pred_wfv
}, index = test.index)
df_pred_wfv.tail()

In [ ]:
fig = px.line(
    data_frame= df_pred_wfv,
    title = "Walk forward Validation Prediction and Actual Test data",
    labels= {"value" : "PM2.5"}
)

fig.show()

In [ ]:
# Evaluation of the Walk forward Validation Prediction

mse = mean_squared_error(test, y_pred_wfv)
mae = mean_absolute_error(test, y_pred_wfv)

print(f"AR Model - MSE: {mse:.2f}")
print(f"AR Model - MAE: {mae:.2f}")

# ARIMA MODEL

In [ ]:
# Fit ARIMA
arima_model = ARIMA(train, order=(2, 1, 3))  # d=1 for first-order differencing
arima_result = arima_model.fit()

In [ ]:
# Forecast
y_pred_test = arima_result.predict(test.index.min(), test.index.max())
y_pred_test.head()



In [ ]:
# Evaluation
mse = mean_squared_error(test, y_pred_test)
mae = mean_absolute_error(test, y_pred_test)

print(f"ARIMA Model - MSE: {mse:.2f}")
print(f"ARIMA Model - MAE: {mae:.2f}")


In [ ]:
# Plot
plt.figure(figsize=(15, 5))
plt.plot(test, label="Actual", color='black')
plt.plot(y_pred_test, label="Forecast", color='green')
plt.title("ARIMA Model Forecast vs Actual")
plt.xlabel("Date")
plt.ylabel("PM2.5")
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Walk Forward Validation

y_pred_wfv = pd.Series()
history = train.copy()
for i in range(len(test)):
     model = ARIMA(history, order=(2, 1, 3)).fit()
     next_pred = model.forecast()
     y_pred_wfv = pd.concat([y_pred_wfv, next_pred])
     history = pd.concat([history, test[next_pred.index]])

In [ ]:
df_pred_wfv = pd.DataFrame({
            'y_test':test,
            'y_pred':y_pred_wfv
}, index = test.index)

df_pred_wfv.tail()

In [ ]:
fig = px.line(
    data_frame= df_pred_wfv,
    title = "Walk forward Validation Prediction and Actual Test data",
    labels= {"value" : "PM2.5"}
)

fig.show()

In [ ]:
# Check out necessary charts

arima_result.plot_diagnostics(figsize=(15, 8))
plt.show()

In [ ]:
# Evaluation of the Walk forward Validation Prediction

mse = mean_squared_error(test, y_pred_wfv)
mae = mean_absolute_error(test, y_pred_wfv)

print(f"AR Model - MSE: {mse:.2f}")
print(f"AR Model - MAE: {mae:.2f}")

In [ ]:
# Get current timestamp in ISO format, replacing ':' to avoid file path issues
timestamp = datetime.now().isoformat(timespec='seconds').replace(":", "-")

# Define the result folder (make sure it exists)
result_dir = "../results/"
os.makedirs(result_dir, exist_ok=True)

# Define filenames
model_filename = f"{timestamp}_arima_model.pkl"
wfv_filename = f"{timestamp}_arima_wfv.csv"

# Save model
joblib.dump(arima_result, os.path.join(result_dir, model_filename))

# Save walk-forward DataFrame (assuming it's named `wfv_df`)
df_pred_wfv.to_csv(os.path.join(result_dir, wfv_filename), index=True)

# SARIMA (Seasonal ARIMA) Model

In [ ]:
# Fit SARIMA
sarima_model = SARIMAX(train,
                       order=(2, 1, 3),
                       seasonal_order=(1, 0, 1, 28),
                       enforce_stationarity=False,
                       enforce_invertibility=False)

sarima_result = sarima_model.fit()

# Best model:  ARIMA(2,1,3)(1,0,1)[28]

In [ ]:
# Forecast
y_sarima_pred = sarima_result.predict(test.index.min(), test.index.max())
y_sarima_pred.head(10)


In [ ]:
# Baseline Model Metrics
mean = train.mean()
y_mean_pred = len(train) * [mean]

# MSE and MAE of the Baseline model
mse_baseline = mean_squared_error(train, y_mean_pred)
mae_baseline = mean_absolute_error(train, y_mean_pred)


print(f"MSE of the baseline model: {round(mse_baseline, 2)}")
print(f"MAE of the baseline model: {round(mae_baseline, 2)}")

In [ ]:
# Evaluation
mse = mean_squared_error(test, y_sarima_pred)
mae = mean_absolute_error(test, y_sarima_pred)

print(f"SARIMA Model - MSE: {mse:.2f}")
print(f"SARIMA Model - MAE: {mae:.2f}")

In [ ]:
# Residual graph
fig, ax = plt.subplots(figsize = (15, 5))
sarima_result.resid.plot(ax = ax)
plt.title("Residual Plot")
plt.xlabel("Date")
plt.show()

In [ ]:
# Residual Histogram
sarima_result.resid.plot(
    kind = "hist", 
    xlabel = "Residual",
    ylabel = "frequency",
    title = "Residual Distribution"
)

In [ ]:
# DataFrame for Actual Test data and Predictions

df_pred_test = pd.DataFrame({
            'y_test':test,
            'y_pred':y_sarima_pred
}, index = test.index)
df_pred_test.tail()

In [ ]:
# Plot results
fig = px.line(
    data_frame= df_pred_test,
    title = "Prediction vs Actual Test data ",
    labels= {"value" : "PM2.5"}
)
fig.show()

In [ ]:
# Walk Forward Validation

y_pred_wfv = pd.Series()
history = train.copy()
for i in range(len(test)):
     model = SARIMAX(history, order=(2, 1, 3), seasonal_order=(1, 0, 1, 28)).fit()
     next_pred = model.forecast()
     y_pred_wfv = pd.concat([y_pred_wfv, next_pred])
     history = pd.concat([history, test[next_pred.index]])


In [ ]:
df_pred_wfv = pd.DataFrame({
            'y_test':test,
            'y_pred':y_pred_wfv
}, index = test.index)
df_pred_wfv.tail()

In [ ]:
fig = px.line(
    data_frame= df_pred_wfv,
    title = "Walk forward Validation Prediction and Actual Test data",
    labels= {"value" : "PM2.5"}
)

fig.show()

In [ ]:
# Evaluation of the Walk forward Validation Prediction

mse = mean_squared_error(test, y_pred_wfv)
mae = mean_absolute_error(test, y_pred_wfv)

print(f"AR Model - MSE: {mse:.2f}")
print(f"AR Model - MAE: {mae:.2f}")

# Saving Best Performing Model (SARIMA)

- save the model as .pkl
- save the summary log as .txt
- save the work forward validation DataFrame as .csv

In [ ]:
# Get current timestamp in ISO format, replacing ':' to avoid file path issues
timestamp = datetime.now().isoformat(timespec='seconds').replace(":", "-")

# Define the result folder (make sure it exists)
result_dir = "../results/"
os.makedirs(result_dir, exist_ok=True)

# Define filenames
model_filename = f"{timestamp}_sarima_model.pkl"
wfv_filename = f"{timestamp}_walk_forward_results.csv"
summary_filename = f"{timestamp}_sarima_summary.txt"

# Save model
joblib.dump(sarima_result, os.path.join(result_dir, model_filename))

# Save walk-forward DataFrame (assuming it's named `wfv_df`)
df_pred_wfv.to_csv(os.path.join(result_dir, wfv_filename), index=True)

# Save model summary
with open(os.path.join(result_dir, summary_filename), "w") as f:
    f.write(str(sarima_result.summary()))


In [ ]:
# Get all SARIMA model files
model_files = sorted(glob.glob("../results/*_model.pkl"))

# Model filepaths
filepaths = [file for file in model_files]

# Latest model file
actual = filepaths[-1]

# Load model
model = joblib.load(actual)

# Test model summary
model.summary()

In [ ]:
# Load Latest WFV DataFrame
wfv_files = sorted(glob.glob("../results/*_results.csv"))

# CSV filepaths
filepaths = [file for file in wfv_files]

# Latest CSV file
actual = filepaths[-1]

# load csv into DataFrame
wfv_df = pd.read_csv(actual, index_col= "date", parse_dates= True)

print(wfv_df.head())

fig = px.line(
    data_frame= wfv_df,
    title = "Walk forward Validation Prediction and Actual Test data",
    labels= {"value" : "PM2.5"}
)

fig.show()

In [ ]:
# Check out necessary charts

model.plot_diagnostics(figsize=(15, 8))
plt.show()